# Vizualize Landsat

In [ ]:
from osgeo import gdal
from pathlib import Path
import rasterio
from rasterio.features import shapes
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from  matplotlib.colors import LinearSegmentedColormap
import contextily as cx
from shapely.geometry import Point, Polygon

In [ ]:
dataset = gdal.Open("data/LC08_L2SP_194027_20230813_20230819_02_T1_ST_B10.TIF", gdal.GA_ReadOnly)
band = dataset.GetRasterBand(1) # note: band no. starting from 1 not 0
arr = band.ReadAsArray()
plt.imshow(arr)

In [ ]:
# get projection
print(dataset.GetProjection())

In [ ]:
# get corners
ulx, xres, xskew, uly, yskew, yres  = dataset.GetGeoTransform()
lrx = ulx + (dataset.RasterXSize * xres)
lry = uly + (dataset.RasterYSize * yres)
print(f"upper left corner {ulx} {uly}")
print(f"lower right corner {lrx} {lry}")

In [ ]:
# get coordinates of Konstanz
cities = gpd.read_file("data/OD_AX_Kommunales_Gebiet/AX_KommunalesGebiet.shp")
city = cities.loc[cities.Name == "Konstanz", "geometry"].to_crs("EPSG:32632").bounds
print(cities.loc[cities.Name == "Konstanz", "geometry"].to_crs("EPSG:32632").bounds)

In [ ]:
cities.loc[cities.Name == "Konstanz", "geometry"].to_crs("EPSG:32632")

In [ ]:
# crop to Konstanz
upper_left_x = 506402.693
upper_left_y = 5289935
lower_right_x = 516367.261
lower_right_y = 5277858
window = (upper_left_x,upper_left_y,lower_right_x,lower_right_y)

for file in Path('data').glob('*B10.TIF'):
    # translate file
    gdal.Translate(str(file.parents[0] / f"{file.stem}_sm.TIF"), file, projWin = window)

    # create geojson
    tmp = rasterio.open(str(file.parents[0] / f"{file.stem}_sm.TIF")).meta
    c = str(tmp['crs'])
    c_s = c.split(':')

    mask = None
    with rasterio.open(str(file.parents[0] / f"{file.stem}_sm.TIF")) as src:
        image = src.read(1) # first band
        results = (
            {'properties': {'temp': v}, 'geometry': s}
            for i, (s, v)
            in enumerate(shapes(image, mask=mask, transform=tmp['transform']))
        )

    tmp2 = gpd.GeoDataFrame.from_features(list(results), crs=c)
    # filter
    tmp2 = tmp2.loc[tmp2.intersects(cities.loc[cities.Name == "Konstanz", "geometry"].to_crs("EPSG:32632").loc[126])]
    tmp2.to_file(str(file.parents[0] / f"{file.stem}_sm.geojson"), driver='GeoJSON')

In [ ]:
# plot figures

for file in Path('data').glob('*B10_sm.geojson'):
    # get year
    year = str(file)[22:26]

    # read temps
    temps = gpd.read_file(file)

    # plot temps
    cmap=LinearSegmentedColormap.from_list('rg',["g", "r"], N=256)

    fig, ax = plt.subplots()
    ax = temps.to_crs(epsg=3857).plot(column='temp', alpha=0.7, cmap=cmap,scheme='quantiles', ax=ax) # figsize=(10,3)
    cx.add_basemap(ax, attribution = False)

    plt.axis('off')
    plt.savefig(f'figures/landsat_konstanz_{year}.pdf', bbox_inches='tight')
    #plt.show()

In [ ]:
# crop to Marktstätte
upper_left_x = 513274.66299081355
upper_left_y = 5278720.103245781
lower_right_x = 513356.19580278266
lower_right_y = 5278644.798902255
window = (upper_left_x,upper_left_y,lower_right_x,lower_right_y)

# EPSG:3857 -> EPSG:32632
# top left 513274.66299081355 5278720.103245781
# bottom right 513356.19580278266 5278644.798902255

for file in Path('data').glob('*B10.TIF'):
    # translate file
    gdal.Translate(str(file.parents[0] / f"{file.stem}_xs.TIF"), file, projWin = window)

    # create geojson
    tmp = rasterio.open(str(file.parents[0] / f"{file.stem}_xs.TIF")).meta
    c = str(tmp['crs'])
    c_s = c.split(':')

    mask = None
    with rasterio.open(str(file.parents[0] / f"{file.stem}_xs.TIF")) as src:
        image = src.read(1) # first band
        results = (
            {'properties': {'temp': v}, 'geometry': s}
            for i, (s, v)
            in enumerate(shapes(image, mask=mask, transform=tmp['transform']))
        )

    tmp2 = gpd.GeoDataFrame.from_features(list(results), crs=c)
    # filter
    tmp2 = tmp2.loc[tmp2.intersects(cities.loc[cities.Name == "Konstanz", "geometry"].to_crs("EPSG:32632").loc[126])]
    tmp2.to_file(str(file.parents[0] / f"{file.stem}_xs.geojson"), driver='GeoJSON')

In [ ]:
# create bounding box
# top left 513274.66299081355 5278720.103245781
# bottom right 513356.19580278266 5278644.798902255

upper_left_x = 513274.66299081355
upper_left_y = 5278720.103245781
lower_right_x = 513356.19580278266
lower_right_y = 5278644.798902255

np1 = (upper_left_x, upper_left_y)
np2 = (upper_left_x, lower_right_y)
np3 = (lower_right_x, lower_right_y)
np4 = (lower_right_x, upper_left_y)

bb_polygon = Polygon([np1, np2, np3, np4])

In [ ]:
# plot figures
for file in Path('data').glob('*B10_sm.geojson'):
    # get year
    year = str(file)[22:26]

    # read temps
    temps = gpd.read_file(file)

    # calculate quantiles
    temps["quantile"] = pd.qcut(temps["temp"], 10, labels=False)

    # plot temps
    cmap=LinearSegmentedColormap.from_list('rg',["g", "r"], N=256)

    fig, ax = plt.subplots()
    ax = temps.to_crs(epsg=3857).loc[temps.intersects(bb_polygon)].plot(column='quantile', alpha=0.7, cmap=cmap, vmin=0, vmax=9, edgecolor='white', linewidth=2, ax=ax) # figsize=(10,3)
    cx.add_basemap(ax, attribution=False)

    plt.axis('off')
    plt.savefig(f'figures/landsat_marktstaette_{year}.pdf', bbox_inches='tight')
    #plt.show()